<a href="https://colab.research.google.com/github/MostakChy055/Video-Colorization/blob/main/Standard_Edge_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install opencv-contrib-python

In [ ]:
# this is a code to get a pre-trained github file
!wget -O model.yml.gz "https://github.com/opencv/opencv_extra/blob/5e3a56880fb115e757855f8e01e744c154791144/testdata/cv/ximgproc/model.yml.gz"

--2024-07-17 14:18:58--  https://github.com/opencv/opencv_extra/blob/5e3a56880fb115e757855f8e01e744c154791144/testdata/cv/ximgproc/model.yml.gz
Resolving github.com (github.com)... 140.82.121.4
Connecting to github.com (github.com)|140.82.121.4|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘model.yml.gz’

model.yml.gz            [ <=>                ] 158.75K  --.-KB/s    in 0.03s   

2024-07-17 14:18:58 (5.30 MB/s) - ‘model.yml.gz’ saved [162564]



In [ ]:
import os
import cv2
import gzip
import requests

model_path = "/content/model.yml.gz"
model_url = "https://github.com/opencv/opencv_extra/blob/5e3a56880fb115e757855f8e01e744c154791144/testdata/cv/ximgproc/model.yml.gz"
print("Model path exists:", os.path.exists(model_path))

# Download the model file using requests to handle potential download issues
response = requests.get(model_url + "?raw=true")  # Use ?raw=true to get the raw file
response.raise_for_status()  # Check for download errors

with open(model_path, 'wb') as f:
    f.write(response.content)

# Extract the compressed model file
with gzip.open(model_path, 'rb') as f_in:
    with open('/content/model.yml', 'wb') as f_out:
        f_out.write(f_in.read())

# Use the extracted model file path
extracted_model_path = "/content/model.yml"
pDollar = cv2.ximgproc.createStructuredEdgeDetection(extracted_model_path)

Model path exists: True


In [ ]:
import numpy as np

src = cv2.imread("/content/GKujq.jpg")
image = src
image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
image = image.astype(np.float32) / 255.0

edges = pDollar.detectEdges(image)

edges = (edges * 255).astype(np.uint8)
cv2.imwrite("/content/edges_detected.jpg", edges)

error: OpenCV(4.8.0) /io/opencv_contrib/modules/ximgproc/src/structured_edge_detection.cpp:537: error: (-215:Assertion failed) _src.type() == CV_32FC3 in function 'detectEdges'


In [ ]:
from sklearn.cluster import KMeans

binary_edges = edges > 50
num_labels, labels_im = cv2.connectedComponents(binary_edges.astype(np.uint8))

# Split segments based on diversity and pixel count constraints
diversity_threshold = 0.1
min_pixels = 25
new_labels = labels_im.copy()

def calculate_diversity(segment, image):
    pixel_values = image[segment]
    mean_value = np.mean(pixel_values)
    diversity = np.mean((pixel_values - mean_value) ** 2)
    return diversity

for label in range(1, num_labels):
    segment = (labels_im == label)
    if np.sum(segment) < min_pixels:
        continue
    if calculate_diversity(segment, image) > diversity_threshold:
        coords = np.column_stack(np.where(segment))
        if len(coords) > 1:
            kmeans = KMeans(n_clusters=2, random_state=0).fit(coords)
            for idx, coord in enumerate(coords):
                new_labels[coord[0], coord[1]] = num_labels + kmeans.labels_[idx] + 1
            num_labels += 1

# Calculate and display Segment Mean Values (SMV)
smv_image = np.zeros_like(image)
for label in range(1, num_labels):
    segment = (new_labels == label)
    if np.sum(segment) >= min_pixels:
        smv_image[segment] = np.mean(image[segment], axis=0)

# Save and display results
smv_image = (smv_image * 255).astype(np.uint8)
cv2.imwrite("/content/final.jpg", smv_image)

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


True